# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 17 2022 10:04AM,244355,15,30010020,"Alliance Pharma, Inc.",Released
1,Aug 17 2022 10:04AM,244355,15,30010021,"Alliance Pharma, Inc.",Released
2,Aug 17 2022 10:04AM,244355,15,30010023,"Alliance Pharma, Inc.",Released
3,Aug 17 2022 10:04AM,244355,15,30010018,"Alliance Pharma, Inc.",Released
4,Aug 17 2022 10:04AM,244355,15,30010019,"Alliance Pharma, Inc.",Released
5,Aug 17 2022 10:04AM,244355,15,30010022,"Alliance Pharma, Inc.",Released
6,Aug 17 2022 10:04AM,244355,15,30010024,"Alliance Pharma, Inc.",Released
7,Aug 17 2022 10:04AM,244355,15,30010025,"Alliance Pharma, Inc.",Released
8,Aug 17 2022 10:04AM,244355,15,30010026,"Alliance Pharma, Inc.",Released
9,Aug 17 2022 10:04AM,244355,15,30010027,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,244351,Released,20
36,244352,Released,5
37,244353,Released,3
38,244354,Released,4
39,244355,Released,25


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244351,NaN,20.0
244352,NaN,5.0
244353,NaN,3.0
244354,NaN,4.0
244355,NaN,25.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244256,0.0,25.0
244259,12.0,0.0
244261,0.0,1.0
244268,1.0,0.0
244274,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244256,0,25
244259,12,0
244261,0,1
244268,1,0
244274,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244256,0,25
1,244259,12,0
2,244261,0,1
3,244268,1,0
4,244274,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244256,,25
1,244259,12,
2,244261,,1
3,244268,1,
4,244274,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc."
25,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation
29,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc."
32,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc."
37,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc."
46,Aug 17 2022 9:55AM,244351,20,"ACI Healthcare USA, Inc."
66,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc."
67,Aug 17 2022 9:06AM,244343,10,Emerginnova
72,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc."
77,Aug 17 2022 8:49AM,244340,10,TherapeuticsMD Elli Sample


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",,25
1,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,,4
2,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc.",,3
3,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc.",,5
4,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc.",,9
5,Aug 17 2022 9:55AM,244351,20,"ACI Healthcare USA, Inc.",,20
6,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",,1
7,Aug 17 2022 9:06AM,244343,10,Emerginnova,,5
8,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc.",,5
9,Aug 17 2022 8:49AM,244340,10,TherapeuticsMD Elli Sample,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",25,
1,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,4,
2,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc.",3,
3,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc.",5,
4,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc.",9,
5,Aug 17 2022 9:55AM,244351,20,"ACI Healthcare USA, Inc.",20,
6,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",1,
7,Aug 17 2022 9:06AM,244343,10,Emerginnova,5,
8,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc.",5,
9,Aug 17 2022 8:49AM,244340,10,TherapeuticsMD Elli Sample,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",25,
1,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,4,
2,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc.",3,
3,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc.",5,
4,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc.",9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",25.0,NaN
1,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,4.0,NaN
2,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc.",3.0,NaN
3,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc.",5.0,NaN
4,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc.",9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",25.0,0.0
1,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,4.0,0.0
2,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc.",3.0,0.0
3,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc.",5.0,0.0
4,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc.",9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4642269,105.0,1.0
102,244320,1.0,0.0
15,244355,25.0,0.0
16,977112,4.0,0.0
17,488643,2.0,0.0
19,244328,14.0,28.0
20,1954429,53.0,13.0
21,732959,2.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4642269,105.0,1.0
1,102,244320,1.0,0.0
2,15,244355,25.0,0.0
3,16,977112,4.0,0.0
4,17,488643,2.0,0.0
5,19,244328,14.0,28.0
6,20,1954429,53.0,13.0
7,21,732959,2.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,105.0,1.0
1,102,1.0,0.0
2,15,25.0,0.0
3,16,4.0,0.0
4,17,2.0,0.0
5,19,14.0,28.0
6,20,53.0,13.0
7,21,2.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,102,Released,1.0
2,15,Released,25.0
3,16,Released,4.0
4,17,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,17,19,20,21
Status,,,,,,,,
Executing,1.0,0.0,0.0,0.0,0.0,28.0,13.0,1.0
Released,105.0,1.0,25.0,4.0,2.0,14.0,53.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,17,19,20,21
0,Executing,1.0,0.0,0.0,0.0,0.0,28.0,13.0,1.0
1,Released,105.0,1.0,25.0,4.0,2.0,14.0,53.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,17,19,20,21
0,Executing,1.0,0.0,0.0,0.0,0.0,28.0,13.0,1.0
1,Released,105.0,1.0,25.0,4.0,2.0,14.0,53.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()